In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [27]:
data = pd.read_csv('hate-speech-dataset-master/annotations_metadata.csv')

In [28]:
data

,file_id,user_id,subforum_id,num_contexts,label
0,12834217_1,572066,1346,0,noHate
1,12834217_2,572066,1346,0,noHate
2,12834217_3,572066,1346,0,noHate
3,12834217_4,572066,1346,0,hate
4,12834217_5,572066,1346,0,noHate
...,...,...,...,...,...
10939,33676864_5,734541,1388,0,noHate
10940,33677019_1,735154,1388,0,noHate
10941,33677019_2,735154,1388,0,noHate
10942,33677053_1,572266,1388,0,hate


In [29]:
data.describe()

,user_id,subforum_id,num_contexts
count,10944.000000,10944.000000,10944.000000
mean,586478.108918,1369.417306,0.229349
std,30995.245917,17.169674,1.121689
min,572033.000000,1345.000000,0.000000
25%,573558.000000,1354.000000,0.000000
50%,577024.000000,1371.000000,0.000000
75%,586751.000000,1387.000000,0.000000
max,796449.000000,1397.000000,32.000000


In [30]:
data.isnull().sum()

file_id         0
user_id         0
subforum_id     0
num_contexts    0
label           0
dtype: int64

In [31]:
data.dtypes

file_id         object
user_id          int64
subforum_id      int64
num_contexts     int64
label           object
dtype: object

In [32]:
data['label'].unique

<bound method Series.unique of 0        noHate
1        noHate
2        noHate
3          hate
4        noHate
          ...  
10939    noHate
10940    noHate
10941    noHate
10942      hate
10943    noHate
Name: label, Length: 10944, dtype: object>

In [33]:

encoder = LabelEncoder()
data['label']=encoder.fit_transform(data['label'])

In [34]:
data['label'].unique

<bound method Series.unique of 0        2
1        2
2        2
3        0
4        2
        ..
10939    2
10940    2
10941    2
10942    0
10943    2
Name: label, Length: 10944, dtype: int32>

here 2 means no-hate and 0 means hate

In [36]:
# Prepare features and target variable
X = data[[ 'num_contexts']]
y = data['label']

In [37]:
X.shape,y.shape

((10944, 1), (10944,))

In [35]:
vectorizer = TfidfVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(X)
X_vectorized
# Initialize Logistic Regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Perform cross-validation
cv_scores = cross_val_score(model, X_vectorized, y, cv=5)

# Display cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


ValueError: Found input variables with inconsistent numbers of samples: [1, 10944]

In [ ]:



def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove special characters, URLs, and mentions
    text = re.sub(r'http\S+|www\S+|@[^\s]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into text
    text = ' '.join(tokens)
    
    return text


In [ ]:
# Preprocess the tweet text
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sivac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#tweet_text = 'Only among Muslims can one find someone proudly cooing that a rapist, murderer, and misogynist'
tweet_text = 'hi guys i am siva i have build this model to detect comment '
#tweet_text = "I love spending time with my family. #happy"
#tweet_text = "I can't believe he said that to me. So rude!"

preprocessed_tweet = preprocess_text(tweet_text)

# Vectorize the preprocessed tweet using the same vectorizer used during training
vectorized_tweet = vectorizer.transform([preprocessed_tweet])

# Make prediction probabilities on the vectorized tweet
predicted_probabilities = model.predict_proba(vectorized_tweet)[0]

# Get the category with the highest probability
predicted_category_index = np.argmax(predicted_probabilities)

# Map predicted category index to its corresponding label reflecting the level of abuse
category_mapping = {
    2 : "abusive tweets",
    0 : "Not abusive"
}
predicted_label = category_mapping[predicted_category_index]

# Print the predicted label
print("Predicted level of abuse:", predicted_label)


Predicted level of abuse: abusive tweets
